This notebooks investigates the coordinate transformations from cylindrical

In [59]:
import sympy as sp
from typing import *
from itertools import combinations_with_replacement, product

In [2]:
# formulas taken from http://eng-web1.eng.famu.fsu.edu/~dommelen/pdes/style_a/nt_trans.html#note:trans
def compute_first_order_transformation(
    u: sp.Function, 
    xs: List[sp.Symbol], 
    zs: List[sp.Symbol], 
    x_o_zs: List[sp.Expr],
    z_o_xs: List[sp.Expr]) -> Dict[sp.Function, sp.Expr]:
    """Builds a dictionary of the of the first order coordinate transformations"""
    
    check_consistency(xs, x_o_zs)
    check_consistency(zs, z_o_xs)
    
    du_dxs = {}
    for x in xs:
        du_dx = 0
        for z_o_x, z in zip(z_o_xs, zs):
            dz_dx = sp.diff(z_o_x, x)
            du_dz = u(*zs).diff(z)
            du_dx += du_dz * dz_dx
        
        du_dx = make_substitutions(du_dx, xs, x_o_zs)
        du_dxs[u(*xs).diff(x)] = du_dx.simplify()
        
    return du_dxs


def check_consistency(xs: List[sp.Symbol], x_o_zs: List[sp.Expr]):
    for x_o_z, x in zip(x_o_zs, xs):
        _, symbols = x_o_z.as_terms()
        if x in symbols:
            raise ValueError(f"Transformation relations cannot contain the original coordinate. Found {x} in {x_o_z}.")


def make_substitutions(f: sp.Expr, xs: List[sp.Symbol], x_o_zs: List[sp.Expr]) -> sp.Expr:
    f_with_substitutions = f
    for x_o_z, x in zip(x_o_zs, xs):
        f_with_substitutions = f_with_substitutions.subs(x, x_o_z)
        
    return f_with_substitutions

def compute_second_order_transformations(
    u: sp.Function, 
    xs: List[sp.Symbol], 
    zs: List[sp.Symbol],
    x_o_zs: List[sp.Expr],
    z_o_xs: List[sp.Expr]) -> Dict[sp.Function, sp.Expr]:
    
    check_consistency(xs, x_o_zs)
    check_consistency(zs, z_o_xs)
    
    d2u_dxidxjs = {}
    for xi, xj in combinations_with_replacement(xs, 2):
        d2u_dxidxj = 0
        for zk_o_x, zk in zip(z_o_xs, zs):
            du_dzk = u(*zs).diff(zk)
            d2zk_dxidxj = sp.diff(zk_o_x, xi, xj)
            d2u_dxidxj += du_dzk * d2zk_dxidxj
            second_order_terms = 0
            for zl_o_x, zl in zip(z_o_xs, zs):
                d2u_dzkdzl = u(*zs).diff(zk, zl)
                dzl_dxj = zl_o_x.diff(xj)
                second_order_terms += d2u_dzkdzl * dzl_dxj
            
            dzk_dxi = zk_o_x.diff(xi) 
            d2u_dxidxj += second_order_terms * dzk_dxi
            
        d2u_dxidxj = make_substitutions(d2u_dxidxj, xs, x_o_zs)
        d2u_dxidxjs[u(*xs).diff(xi, xj)] = d2u_dxidxj.simplify()
    
    return d2u_dxidxjs

try:
    x = sp.symbols("x")
    check_consistency([x], [x+1])
except ValueError as e:
    print("Found value error as expected.")
    print(e)

Found value error as expected.
Transformation relations cannot contain the original coordinate. Found x in x + 1.


In [3]:
#test with cartesian to cylindrical conversion
x, y, rho, th = sp.symbols(r"x y \rho \theta")
u = sp.Function("u")
rho_exp = sp.sqrt(x**2 + y**2)
th_exp = sp.atan2(y, x)
x_exp = rho * sp.cos(th)
y_exp = rho * sp.sin(th)
sot = compute_second_order_transformations(u, [x, y], [rho, th], [x_exp, y_exp], [rho_exp, th_exp])
assert len(sot) == 3
(sot[u(x,y).diff(x, x)] + sot[u(x,y).diff(y, y)]).simplify()

(\rho**2*Derivative(u(\rho, \theta), (\rho, 2)) + sqrt(\rho**2)*Derivative(u(\rho, \theta), \rho) + Derivative(u(\rho, \theta), (\theta, 2)))/\rho**2

In [4]:
# defined the symbolic variables and function
z, xi, rho, a, p, r, f, theta, phi, kappa, Nu_D = sp.symbols(r"z xi \rho a p r f \theta \phi \kappa \mathrm{Nu}_{D}")
th = sp.Function("\Theta")

In [5]:
r_exp = rho * (1 - p*xi/a) * (1 - f * sp.sin(phi))
r_exp

\rho*(1 - p*xi/a)*(-f*sin(\phi) + 1)

In [6]:
z_exp = xi * 1

In [7]:
xi_exp = z * 1

In [8]:
theta_exp = phi*1

In [9]:
phi_exp = theta*1

In [10]:
rho_exp = sp.solve(r_exp-r, rho)[0].subs(xi, xi_exp).subs(phi, phi_exp)
rho_exp

a*r/(-a*f*sin(\theta) + a + f*p*z*sin(\theta) - p*z)

In [11]:
d1 = compute_first_order_transformation(th, [r, z, theta], [rho, xi, phi], [r_exp, z_exp, theta_exp], [rho_exp, xi_exp, phi_exp])
d2 = compute_second_order_transformations(th, [r, z, theta], [rho, xi, phi], [r_exp, z_exp, theta_exp], [rho_exp, xi_exp, phi_exp])

In [142]:
heat_equation_rhs = kappa * (1/rho * d1[th(r, z, theta).diff(r)] + d2[th(r, z, theta).diff(r,r)] + d2[th(r, z, theta).diff(z,z)]).simplify()
heat_equation_rhs

\kappa*(\rho*a**2*(a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi)*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) + \rho*(a - p*xi)**3*(f*sin(\phi) - 1)**3*(\rho**2*p**2*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) + 2*\rho*a*p*Derivative(\Theta(\rho, xi, \phi), \rho, xi) - 2*\rho*p**2*xi*Derivative(\Theta(\rho, xi, \phi), \rho, xi) + 2*\rho*p**2*Derivative(\Theta(\rho, xi, \phi), \rho) + a**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) - 2*a*p*xi*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) + p**2*xi**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2))) - a*(a - p*xi)**2*(f*sin(\phi) - 1)**2*(a**2 - 2*a*p*xi + p**2*xi**2)*Derivative(\Theta(\rho, xi, \phi), \rho))/(\rho*(a - p*xi)**2*(f*sin(\phi) - 1)**2*(a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi))

In [165]:
# manually separate out terms in the above equation to make downstream expansion and simplification easier.
c_exps = []
c_exps.append(rho*a**2*(a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sp.sin(phi) - a - f*p*xi*sp.sin(phi) + p*xi))
c_exps.append(a*(a - p*xi)**2*(f*sp.sin(phi) - 1)**2*(a**2 - 2*a*p*xi + p**2*xi**2))
c_exps.append(rho*(a - p*xi)**3*(f*sp.sin(phi) - 1)**3)
c_exps.append(rho*(a - p*xi)**2*(f*sp.sin(phi) - 1)**2*(a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sp.sin(phi) - a - f*p*xi*sp.sin(phi) + p*xi))
cs = sp.symbols(" ".join((f"c_{i}" for i in range(len(c_exps)))))
simplified_rhs = heat_equation_rhs
for c_exp, c in zip(c_exps, cs):
    simplified_rhs = simplified_rhs.subs(c_exp, c)

print(simplified_rhs)
simplified_rhs

\kappa*(c_0*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) - c_1*Derivative(\Theta(\rho, xi, \phi), \rho) + c_2*(\rho**2*p**2*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) + 2*\rho*a*p*Derivative(\Theta(\rho, xi, \phi), \rho, xi) - 2*\rho*p**2*xi*Derivative(\Theta(\rho, xi, \phi), \rho, xi) + 2*\rho*p**2*Derivative(\Theta(\rho, xi, \phi), \rho) + a**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) - 2*a*p*xi*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) + p**2*xi**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2))))/c_3


\kappa*(c_0*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) - c_1*Derivative(\Theta(\rho, xi, \phi), \rho) + c_2*(\rho**2*p**2*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) + 2*\rho*a*p*Derivative(\Theta(\rho, xi, \phi), \rho, xi) - 2*\rho*p**2*xi*Derivative(\Theta(\rho, xi, \phi), \rho, xi) + 2*\rho*p**2*Derivative(\Theta(\rho, xi, \phi), \rho) + a**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) - 2*a*p*xi*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) + p**2*xi**2*Derivative(\Theta(\rho, xi, \phi), (xi, 2))))/c_3

In [166]:
def get_coefficients(expr, terms: List) -> Dict:
    coefficients = {}
    expaned_expr = expr.expand()
    for term in terms:
        coefficient = expaned_expr.coeff(term)
        coefficient.simplify()
        coefficients[term] = coefficient

    return coefficients

variables = rho, xi, phi
first_order_terms = [th(*variables).diff(variable) for variable in variables]
second_order_terms = [first_order_term.diff(variable) for first_order_term, variable in product(first_order_terms, variables)]
rhs_coefficients = get_coefficients(simplified_rhs, first_order_terms + second_order_terms)
print(rhs_coefficients)

{Derivative(\Theta(\rho, xi, \phi), \rho): 2*\kappa*\rho*c_2*p**2/c_3 - \kappa*c_1/c_3, Derivative(\Theta(\rho, xi, \phi), xi): 0, Derivative(\Theta(\rho, xi, \phi), \phi): 0, Derivative(\Theta(\rho, xi, \phi), (\rho, 2)): \kappa*\rho**2*c_2*p**2/c_3 + \kappa*c_0/c_3, Derivative(\Theta(\rho, xi, \phi), \rho, xi): 2*\kappa*\rho*a*c_2*p/c_3 - 2*\kappa*\rho*c_2*p**2*xi/c_3, Derivative(\Theta(\rho, xi, \phi), \phi, \rho): 0, Derivative(\Theta(\rho, xi, \phi), (xi, 2)): \kappa*a**2*c_2/c_3 - 2*\kappa*a*c_2*p*xi/c_3 + \kappa*c_2*p**2*xi**2/c_3, Derivative(\Theta(\rho, xi, \phi), \phi, xi): 0, Derivative(\Theta(\rho, xi, \phi), (\phi, 2)): 0}


In [167]:
print("Coefficients for each term on the right-hand-side of the heat equation.")
for term, simplified_coefficient in rhs_coefficients.items():
    if simplified_coefficient != 0:
        full_coefficient = simplified_coefficient
        for c_exp, c in zip(c_exps, cs):
            full_coefficient = full_coefficient.subs(c, c_exp)
            full_coefficient.simplify()
        print(f"{term}:")
        print(f"\t{full_coefficient}\n")

Coefficients for each term on the right-hand-side of the heat equation.
Derivative(\Theta(\rho, xi, \phi), \rho):
	2*\kappa*\rho*p**2*(a - p*xi)*(f*sin(\phi) - 1)/((a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi)) - \kappa*a/(\rho*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi))

Derivative(\Theta(\rho, xi, \phi), (\rho, 2)):
	\kappa*\rho**2*p**2*(a - p*xi)*(f*sin(\phi) - 1)/((a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi)) + \kappa*a**2/((a - p*xi)**2*(f*sin(\phi) - 1)**2)

Derivative(\Theta(\rho, xi, \phi), \rho, xi):
	2*\kappa*\rho*a*p*(a - p*xi)*(f*sin(\phi) - 1)/((a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi)) - 2*\kappa*\rho*p**2*xi*(a - p*xi)*(f*sin(\phi) - 1)/((a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi))

Derivative(\Theta(\rho, xi, \phi), (xi, 2)):
	\kappa*a**2*(a - p*xi)*(f*sin(\phi) - 1)/((a**2 - 2*a*p*xi + p**2*xi**2)*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + 

In [164]:
# sanity check to see if the 2D cylindrical heat equation is recovered
sp.limit(sp.limit(rhs, p, 0), f, 0).simplify()

\kappa*(\rho*Derivative(\Theta(\rho, xi, \phi), (\rho, 2)) + \rho*Derivative(\Theta(\rho, xi, \phi), (xi, 2)) + Derivative(\Theta(\rho, xi, \phi), \rho))/\rho

In [66]:
# define the boundary surface (except for the ends)
r_surface = r_exp
x_surface = (r_surface * sp.cos(phi))
y_surface = (r_surface * sp.sin(phi))
z_surface = xi
s = sp.Matrix([x_surface.simplify(), y_surface.simplify(), z_surface])
s

Matrix([
[-\rho*(a - p*xi)*(f*sin(\phi) - 1)*cos(\phi)/a],
[-\rho*(a - p*xi)*(f*sin(\phi) - 1)*sin(\phi)/a],
[                                            xi]])

In [67]:
#compute the normal vector (taken from https://en.wikipedia.org/wiki/Normal_(geometry))
dsdxi = s.diff(xi)
dsdphi = s.diff(phi)
n = dsdphi.cross(dsdxi)
n.simplify()
n

Matrix([
[       \rho*(a - p*xi)*(-f*sin(2*\phi) + cos(\phi))/a],
[\rho*(a - p*xi)*(-2*f*sin(\phi)**2 + f + sin(\phi))/a],
[       \rho**2*p*(a - p*xi)*(f*sin(\phi) - 1)**2/a**2]])

In [68]:
# x direction asymptotic
sp.limit(sp.limit(n[0], p, 0), f, 0)

\rho*cos(\phi)

In [69]:
# y direction asymptotic
sp.limit(sp.limit(n[1], p, 0), f, 0)

\rho*sin(\phi)

In [70]:
# z direction asymptotic
sp.limit(sp.limit(n[2], p, 0), f, 0)

0

In [19]:
# define the cartesian coordinate transformations
x_exp = r_exp * sp.cos(phi)
x_exp

\rho*(1 - p*xi/a)*(-f*sin(\phi) + 1)*cos(\phi)

In [20]:
y_exp = r_exp * sp.sin(phi)
y_exp

\rho*(1 - p*xi/a)*(-f*sin(\phi) + 1)*sin(\phi)

In [21]:
z_exp = xi
z_exp

xi

In [22]:
phi_exp = sp.atan2(y, x)
phi_exp

atan2(y, x)

In [23]:
zi_exp = z
zi_exp

z

In [24]:
rho_exp = sp.solve(x_exp-x, rho)[0].subs(phi, phi_exp).subs(xi, xi_exp).simplify()
rho_exp

-a*(x**2 + y**2)/(a*f*y - f*p*y*z - (a - p*z)*sqrt(x**2 + y**2))

In [25]:
fot = compute_first_order_transformation(th, [x, y, z], [rho, phi, xi], [x_exp, y_exp, z_exp], [rho_exp, phi_exp, xi_exp])
fot

{Derivative(\Theta(x, y, z), x): a*(\rho**2*(a - p*xi)*(f*sin(\phi) - 1)**2*(-2*\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2) - 2*a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))*cos(\phi)*Derivative(\Theta(\rho, \phi, xi), \rho) + (\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2*sin(\phi)*Derivative(\Theta(\rho, \phi, xi), \phi))/(\rho*(a - p*xi)*(f*sin(\phi) - 1)*(\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2),
 Derivative(\Theta(x, y, z), y): (\rho**2*(a - p*xi)*(f*sin(\phi) - 1)**2*(\rho*(a - p*xi)*(f*sin(\phi) - 1)*(\rho*(a - p*xi)*(f*sin(\phi) - 1)*sin(\phi) + a*f*sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)) - 2*a*sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*(-\rho*f*p*x

In [26]:
gradient = sp.Matrix([fot[th(x, y, z).diff(x)], fot[th(x, y, z).diff(y)], fot[th(x, y, z).diff(z)]])
gradient

Matrix([
[                                                                                                                                                                                                                                               a*(\rho**2*(a - p*xi)*(f*sin(\phi) - 1)**2*(-2*\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2) - 2*a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))*cos(\phi)*Derivative(\Theta(\rho, \phi, xi), \rho) + (\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2*sin(\phi)*Derivative(\Theta(\rho, \phi, xi), \phi))/(\rho*(a - p*xi)*(f*sin(\phi) - 1)*(\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2)],
[(\rho**2*(a - p*xi)*(f*sin(\phi) - 1)**2*(\rho*(a - p*xi)*(f*sin(\phi

In [56]:
g1 = sp.refine(gradient[0], sp.Q.positive(a))
g1

a*(\rho**2*(a - p*xi)*(f*sin(\phi) - 1)**2*(-2*\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2) - 2*a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))*cos(\phi)*Derivative(\Theta(\rho, \phi, xi), \rho) + (\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2*sin(\phi)*Derivative(\Theta(\rho, \phi, xi), \phi))/(\rho*(a - p*xi)*(f*sin(\phi) - 1)*(\rho*f*p*xi*(-f*sin(\phi) + 1)*sin(\phi) + a*(-\rho*f*(-f*sin(\phi) + 1)*sin(\phi) + sqrt(\rho**2*(a - p*xi)**2*(f*sin(\phi) - 1)**2/a**2)))**2)

In [52]:
def build_gradient_coefficient_matrix(gradient: sp.Matrix, function: sp.Function, variables: List[sp.Symbol], assumptions: Optional[List] = None) -> sp.Matrix:
    """This function takes a computed gradient and finds the coefficients. Assumes a linear relation between the function's derivatives and the gradient."""
    coefficient_matrix = []
    for component in gradient:
        expanded_component = component.expand()
        coefficient_vector = []
        for variable in variables:
            function_derivative = function(*variables).diff(variable)
            coefficient = expanded_component.coeff(function_derivative, 1)
            if assumptions:
                for assumption in assumptions:
                    coefficient = sp.refine(coefficient, assumption)
            else:
                coefficient = coefficient.simplify()
            coefficient_vector.append(coefficient.refine())
        
        coefficient_matrix.append(coefficient_vector)
    
    return sp.Matrix(coefficient_matrix)

# test with a simple function first
func = sp.Function("F")
grad = sp.Matrix([
    x**2.0 * func(x,y).diff(x) + y*func(x,y).diff(y),
    y * func(x,y).diff(x) + z*func(x,y).diff(y),
])
build_gradient_coefficient_matrix(grad, func, [x, y])

Matrix([
[x**2.0, y],
[     y, z]])

In [57]:
coefficient_matrix = build_gradient_coefficient_matrix(gradient, th, [rho, phi, xi])
print(coefficient_matrix)

Matrix([[a*(-2*\rho*a*f**2*sin(\phi)**2 + 2*\rho*a*f*sin(\phi) + 2*\rho*f**2*p*xi*sin(\phi)**2 - 2*\rho*f*p*xi*sin(\phi) - a*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2))*cos(\phi)/(\rho*a**2*f**3*sin(\phi)**3 - \rho*a**2*f**2*sin(\phi)**2 + \rho*a**2*f*sin(\phi) - \rho*a**2 - 2*\rho*a*f**3*p*xi*sin(\phi)**3 + 2*\rho*a*f**2*p*xi*sin(\phi)**2 - 2*\rho*a*f*p*xi*sin(\phi) + 2*\rho*a*p*xi + \rho*f**3*p**2*xi**2*sin(\phi)**3 - \rho*f**2*p**2*xi**2*sin(\phi)**2 + \rho*f*p**2*xi**2*sin(\phi) - \rho*p**2*xi**2 + 2*a**2*f*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi) - 2*a*f*p*xi*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi)), a*sin(\phi)/(\rho*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi)), 0], [a*(-\rho*a*f*sin(\phi)**2 + \rho*a*sin(\phi) + \rho*f*p*xi*sin(\phi)**2 - \rho*p*xi*sin(\phi) - 2*a*f*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi)**2 + a*f*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2))/(2*\rho*a**2*f**2*sin(\

In [76]:
#check the assymptotic behavior
assymptotic_matrix = coefficient_matrix.subs(f, 0.0).subs(p, 0.0)
assymptotic_matrix 

Matrix([
[sqrt(\rho**2)*cos(\phi)/\rho, -sin(\phi)/\rho, 0],
[\rho*sin(\phi)/sqrt(\rho**2),  cos(\phi)/\rho, 0],
[                           0,               0, 1]])

In [79]:
assymptotic_normal = n.subs(f, 0.0).subs(p, 0.0)
assymptotic_normal /= sp.sqrt(assymptotic_normal.dot(assymptotic_normal))
assymptotic_normal.simplify()
assymptotic_normal.transpose()

Matrix([[\rho*cos(\phi)/sqrt(\rho**2), \rho*sin(\phi)/sqrt(\rho**2), 0]])

In [80]:
(assymptotic_normal.transpose() * assymptotic_matrix)

Matrix([[\rho**2*sin(\phi)**2/\rho**2 + cos(\phi)**2, 0, 0]])

In [81]:
# now print the elements of the matrix for copy-paste
num_rows = coefficient_matrix.shape[0]
num_cols = coefficient_matrix.shape[1]
print("Coefficient Matrix for radial boundary condition")
for i, j in product(range(num_rows), range(num_cols)):
    print(f"A_{i}{j}:")
    print(f"\t{sp.refine(coefficient_matrix[i, j], sp.Q.positive(rho))}\n")

Coefficient Matrix for radial boundary condition
A_00:
	a*(-2*\rho*a*f**2*sin(\phi)**2 + 2*\rho*a*f*sin(\phi) + 2*\rho*f**2*p*xi*sin(\phi)**2 - 2*\rho*f*p*xi*sin(\phi) - a*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2))*cos(\phi)/(\rho*a**2*f**3*sin(\phi)**3 - \rho*a**2*f**2*sin(\phi)**2 + \rho*a**2*f*sin(\phi) - \rho*a**2 - 2*\rho*a*f**3*p*xi*sin(\phi)**3 + 2*\rho*a*f**2*p*xi*sin(\phi)**2 - 2*\rho*a*f*p*xi*sin(\phi) + 2*\rho*a*p*xi + \rho*f**3*p**2*xi**2*sin(\phi)**3 - \rho*f**2*p**2*xi**2*sin(\phi)**2 + \rho*f*p**2*xi**2*sin(\phi) - \rho*p**2*xi**2 + 2*a**2*f*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi) - 2*a*f*p*xi*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi))

A_01:
	a*sin(\phi)/(\rho*(a*f*sin(\phi) - a - f*p*xi*sin(\phi) + p*xi))

A_02:
	0

A_10:
	a*(-\rho*a*f*sin(\phi)**2 + \rho*a*sin(\phi) + \rho*f*p*xi*sin(\phi)**2 - \rho*p*xi*sin(\phi) - 2*a*f*sqrt(\rho**2*(-a + p*xi)**2*(f*sin(\phi) - 1)**2/a**2)*sin(\phi)**2 + a*f*sqrt(\rho**2*

In [82]:
# now print the elements of the matrix for copy-paste
num_rows = n.shape[0]
print("Unscaled Normal Vector")
for i in range(num_rows):
    print(f"n_{i}:")
    print(f"\t{sp.refine(n[i], sp.Q.positive(rho))}\n")

Unscaled Normal Vector
n_0:
	\rho*(a - p*xi)*(-f*sin(2*\phi) + cos(\phi))/a

n_1:
	\rho*(a - p*xi)*(-2*f*sin(\phi)**2 + f + sin(\phi))/a

n_2:
	\rho**2*p*(a - p*xi)*(f*sin(\phi) - 1)**2/a**2

